In [7]:
import gsw
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
import xarray as xr

from acspype import ACSDev
from acspype import ACSTSCor
import acspype.processing as acsproc
import acspype.qaqc as acsqaqc
from acspype.tscor import ACSTSCor

In [8]:
data_filepath = '../dev_tools/test_files/TEST_SHIMADA_202405.nc'
acs = xr.open_dataset(data_filepath)

In [9]:
dev_filepath = '../dev_tools/test_files/ACS-00412_2023-05-10.dev'
dev = ACSDev(dev_filepath)

In [11]:
acs

<xarray.Dataset> Size: 49MB
Dimensions:                            (time: 16622, a_wavelength: 89,
                                        c_wavelength: 89)
Coordinates:
  * time                               (time) datetime64[ns] 133kB 2024-06-03...
  * a_wavelength                       (a_wavelength) float64 712B 401.9 ... ...
  * c_wavelength                       (c_wavelength) float64 712B 401.4 ... ...
Data variables: (12/13)
    latitude                           (time) float64 133kB ...
    longitude                          (time) float64 133kB ...
    raw_internal_temperature           (time) int64 133kB ...
    raw_external_temperature           (time) int64 133kB ...
    a_signal                           (time, a_wavelength) int64 12MB ...
    a_reference                        (time, a_wavelength) int64 12MB ...
    ...                                 ...
    c_reference                        (time, c_wavelength) int64 12MB ...
    elapsed_time                       (time) int64 133kB ...
    sea_water_temperature              (time) float64 133kB ...
    sea_water_electrical_conductivity  (time) float64 133kB ...
    sea_water_practical_salinity       (time) float64 133kB ...
    seawater_state                     (time) float64 133kB ...
Attributes: (12/13)
    acs_serial_number:           ACS412
    acs_device_file:             ACS0412_2023-05-10.DEV
    acs_num_output_wavelengths:  89
    flow_meter_serial_number:    11222239
    flowmeter_kfactor:           617.2
    flowmeter_manufacturer:      Omega
    ...                          ...
    tsg_serial_number:           tsg00001
    tsg_manufacturer:            Atlas Scientific
    tsg_model:                   Industrial Conductivity Probe K1.0
    valve_serial_number:         valve00001
    dataset_description:         This is a curated dataset of ACS and ancilla...
    dataset_owner:               Maria T. Kavanaugh, Oregon State University

In [14]:
acs.acs_serial_number

'ACS412'

In [12]:
dir(dev)

['_ACSDev__build_interp_funcs',
 '_ACSDev__check_parse',
 '_ACSDev__parse_metadata',
 '_ACSDev__parse_offsets',
 '_ACSDev__parse_tbins',
 '_ACSDev__read_dev',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_filepath',
 'a_delta_t',
 'a_offset',
 'a_wavelength',
 'baudrate',
 'c_delta_t',
 'c_offset',
 'c_wavelength',
 'cal_date',
 'delta_t_interp_method',
 'depth_cal_1',
 'depth_cal_2',
 'func_a_delta_t',
 'func_c_delta_t',
 'ical',
 'max_a_difference',
 'max_a_noise',
 'max_a_nonconform',
 'max_c_difference',
 'max_c_noise',
 'max_c_nonconform',
 'max_depth_sd',
 'max_temp_sd',
 'min_a_counts',
 'min_c_counts',
 'min_r_counts',
 'num_tb

In [13]:
dev.serial_number

'ACS-00412'